In [1]:
import pandas as pd
import csv

In [3]:
with open('Champions_Info.csv', mode='r') as infile:
    reader = csv.reader(infile)
    champions_role_dict = {rows[0]:rows[1] for rows in reader}

def AverageDict(dictionary):
    filtered_vals=[v for _, v in dictionary.items() if v != 0]
    return(sum(filtered_vals)/len(filtered_vals))

def orderDict(dictionary):
    return({k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1],reverse=True)})

def GetFirstKey(dictionary):
    return ([key for key in dictionary.keys()][0])

def GetFirstValue(dictionary):
    return ([value for value in dictionary.values()][0])

In [2]:
export_txt_name_list=['Data\challenger_results.txt','Data\grandmaster_results.txt','Data\master_results.txt']
data_paths_list=['Data\challenger_data.csv','Data\grandmaster_data.csv','Data\master_data.csv']
data_path_index=0

for export_txt_name in export_txt_name_list:

    df = pd.read_csv(data_paths_list[data_path_index])
    data_path_index=data_path_index+1  
    winners_columns_names=["Win_1","Win_2","Win_3","Win_4","Win_5"]
    losers_columns_names=["Lose_1","Lose_2","Lose_3","Lose_4","Lose_5"]
    blue_winners_df=pd.DataFrame(columns=winners_columns_names)
    blue_losers_df=pd.DataFrame(columns=losers_columns_names)
    red_winners_df=pd.DataFrame(columns=winners_columns_names)
    red_losers_df=pd.DataFrame(columns=losers_columns_names)

    for row in range(len(df.index)):
        winner_champion_index=0
        loser_champion_index=0
        winner_champion_list=[]
        loser_champion_list=[]
        #Blue team win
        if df['blue_team_win'][row]==1:
            #Blue winners
            while winner_champion_index<5:
                for column in range((len(df.columns)-1)):
                    if column % 2 == 0:
                        if df[df.columns[column]][row]==1:
                            champion_name_cleaned=df.columns[column][:len(df.columns[column])-5]
                            winner_champion_list.append(champion_name_cleaned)
                            winner_champion_index=winner_champion_index+1
                    if winner_champion_index==5:
                        break
            #Red losers
            while loser_champion_index<5:
                for column in range((len(df.columns)-1)):
                    if column % 2 != 0:
                        if df[df.columns[column]][row]==1:
                            champion_name_cleaned=df.columns[column][:len(df.columns[column])-4]
                            loser_champion_list.append(champion_name_cleaned)
                            loser_champion_index=loser_champion_index+1     
                    if loser_champion_index==5:
                        break
            blue_winners_df.loc[len(blue_winners_df.index)]=winner_champion_list
            red_losers_df.loc[len(red_losers_df.index)]=loser_champion_list
        #Red team win
        if df['blue_team_win'][row]==0:
            #Red winners
            while winner_champion_index<5:
                for column in range((len(df.columns)-1)):
                    if column % 2 != 0:
                        if df[df.columns[column]][row]==1:
                            champion_name_cleaned=df.columns[column][:len(df.columns[column])-4]
                            winner_champion_list.append(champion_name_cleaned)
                            winner_champion_index=winner_champion_index+1
                    if winner_champion_index==5:
                        break
            #Blue losers
            while loser_champion_index<5:
                for column in range((len(df.columns)-1)):
                    if column % 2 == 0:
                        if df[df.columns[column]][row]==1:
                            champion_name_cleaned=df.columns[column][:len(df.columns[column])-5]
                            loser_champion_list.append(champion_name_cleaned)
                            loser_champion_index=loser_champion_index+1
                    if loser_champion_index==5:
                        break     
            red_winners_df.loc[len(red_winners_df.index)]=winner_champion_list
            blue_losers_df.loc[len(blue_losers_df.index)]=loser_champion_list

    #Build all winners dataframe
    all_winners=pd.concat([blue_winners_df,red_winners_df], ignore_index=True)
    all_losers=pd.concat([blue_losers_df,red_losers_df],ignore_index=True)

    #Get stats
    all_winners_stats=(pd.Series(all_winners.values.ravel('F')).value_counts()).to_frame().reset_index()
    all_losers_stats=pd.Series(all_losers.values.ravel('F')).value_counts()

    #Get the best 5 champions for each line
    best_top_champions={}
    best_jungle_champions={}
    best_mid_champions={}
    best_sup_champions={}
    best_adc_champions={}

    best_top_champion_loses={}
    best_jungle_champions_loses={}
    best_mid_champions_loses={}
    best_sup_champions_loses={}
    best_adc_champions_loses={}

    champions_reviewed=0
    total_amount_of_games=len(all_winners.index)

    while champions_reviewed!=25:
        index=0
        for winner in all_winners_stats['index']:
            if (champions_role_dict[winner]=='Top') and (len(best_top_champions)<5):
                best_top_champions[winner]=all_winners_stats[0][index]
                best_top_champion_loses[winner]=all_losers_stats[winner]
                champions_reviewed=champions_reviewed+1
                index=index+1
            if (champions_role_dict[winner]=='Mid') and (len(best_mid_champions)<5):
                best_mid_champions[winner]=all_winners_stats[0][index]
                best_mid_champions_loses[winner]=all_losers_stats[winner]
                champions_reviewed=champions_reviewed+1
                index=index+1
            if (champions_role_dict[winner]=='Support') and (len(best_sup_champions)<5):
                best_sup_champions[winner]=all_winners_stats[0][index]
                best_sup_champions_loses[winner]=all_losers_stats[winner]
                champions_reviewed=champions_reviewed+1
                index=index+1
            if (champions_role_dict[winner]=='ADC') and (len(best_adc_champions)<5):
                best_adc_champions[winner]=all_winners_stats[0][index]
                best_adc_champions_loses[winner]=all_losers_stats[winner]
                champions_reviewed=champions_reviewed+1
                index=index+1
            if (champions_role_dict[winner]=='Jungle') and (len(best_jungle_champions)<5):
                best_jungle_champions[winner]=all_winners_stats[0][index]
                best_jungle_champions_loses[winner]=all_losers_stats[winner]
                champions_reviewed=champions_reviewed+1
                index=index+1
            if champions_reviewed==25:
                break
        
    average_top_loses=AverageDict(best_top_champion_loses)
    average_jungle_loses=AverageDict(best_jungle_champions_loses)
    average_mid_loses=AverageDict(best_mid_champions_loses)
    average_adc_loses=AverageDict(best_adc_champions_loses)
    average_sup_loses=AverageDict(best_sup_champions_loses)
    
    #Calculate winrates with relativity correction
    list_of_dictionaries=[best_top_champions, best_jungle_champions, best_mid_champions, best_adc_champions, best_sup_champions]
    list_of_lose_dictionaries=[best_top_champion_loses, best_jungle_champions_loses, best_mid_champions_loses, best_adc_champions_loses, best_sup_champions_loses]
    average_lose_dictionary=[average_top_loses, average_jungle_loses, average_mid_loses, average_adc_loses, average_sup_loses]
    index=0
    for list in list_of_dictionaries:
        for key in list:
            wins=list[key]
            loses = list_of_lose_dictionaries[index][key]
            total= wins+loses
            average_loses=average_lose_dictionary[index]
            list[key]=wins*average_loses*100/(total*loses)
        index=index+1
            
    #order each dictionary from highest winrate to lowest
    best_top_champions=orderDict(best_top_champions)
    best_jungle_champions=orderDict(best_jungle_champions)
    best_mid_champions=orderDict(best_mid_champions)
    best_adc_champions=orderDict(best_adc_champions)
    best_sup_champions=orderDict(best_sup_champions)

    #get best overall composition
    best_overall_champions=(GetFirstKey(best_top_champions),GetFirstKey(best_jungle_champions),GetFirstKey(best_mid_champions),GetFirstKey(best_adc_champions),GetFirstKey(best_sup_champions))
    best_overall_winrates=(GetFirstValue(best_top_champions),GetFirstValue(best_jungle_champions),GetFirstValue(best_mid_champions),GetFirstValue(best_adc_champions),GetFirstValue(best_sup_champions))

    best_overall_composition=dict(zip(best_overall_champions,best_overall_winrates))

    dictionaries_to_export=[best_top_champions,best_jungle_champions,best_mid_champions,best_adc_champions,best_sup_champions,best_overall_composition]

    with open(export_txt_name,"w") as output:
        output.write(str(dictionaries_to_export))